In [1]:
import numpy as np
import os
import skimage.filters as skfilters
import matplotlib.pyplot as plt
import skimage.io as io

from skimage import img_as_uint
from skimage.filters import threshold_otsu


In [2]:
small_files = os.listdir(os.path.join(os.getcwd(), 'small_tifs'))
for i in range(len(small_files)):
    small_files[i] = os.path.join(os.getcwd(), 'small_tifs', small_files[i])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/pgb13/repositories/image-pipelines/jupyter_notebooks/small_tifs'

In [ ]:
small_files
images = [small_files[0], small_files[5], small_files[6]]

In [ ]:
d3mko = io.imread(small_files[2])

In [ ]:
images


In [ ]:
d3mko.shape

In [ ]:
thresh = threshold_otsu(d3mko)

In [ ]:
binary = d3mko > thresh

In [ ]:
thresh = threshold_otsu(d3mko)
binary = d3mko > thresh
fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(d3mko, cmap=plt.cm.gray)
ax[0].set_title('Day 3 mko')
ax[0].axis('off')

ax[1].hist(d3mko.ravel(), bins=256)
ax[1].set_title('Hisotgram')
ax[1].axvline(thresh, color='r')

ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Threshold')
ax[2].axis('off')

plt.show()
io.imsave(os.path.join(os.getcwd(), 'export', 'binary.tif'), img_as_uint(binary))




In [ ]:
plt.imshow(d3mko, cmap=plt.cm.gray)

In [ ]:
plt.imshow(binary, cmap=plt.cm.gray)

In [ ]:
from skimage.filters import try_all_threshold


In [ ]:
#im = d3mko
#fig, ax = try_all_threshold(im, figsize=(10,8))
#plt.show()
#commented out. These can be used to try all of the filtering methods

# Next Section - label image regions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb


In [ ]:

thresh = threshold_otsu(d3mko)
bw = closing(d3mko > thresh, square(3))

cleared = clear_border(bw)

label_image = label(cleared)
image_label_overlay = label2rgb(label_image, image=d3mko)
fig, ax = plt.subplots(figsize=(10,6))
ax.imshow(image_label_overlay)

nuclei = 0


for region in regionprops(label_image):
    if region.area >= 20:
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                 fill=False, edgecolor='red', linewidth=2)
        nuclei += 1
        ax.add_patch(rect)
        
ax.set_axis_off()
plt.tight_layout()
plt.show()
print(nuclei)


In [ ]:
from skimage.filters import threshold_yen, threshold_adaptive, threshold_li, threshold_niblack, threshold_sauvola, threshold_triangle

thresh_list = [threshold_yen, threshold_li, threshold_otsu,
              threshold_niblack, threshold_sauvola, threshold_triangle]

fig, axes = plt.subplots(ncols=1, nrows=len(thresh_list), figsize=(50, 20))
    #ax[0] = plt.subplot(1, 3, 1)
    #ax[1] = plt.subplot(1, 3, 2)
    #ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])
ax = axes.ravel()
for i in range(len(thresh_list)):
    threshold = thresh_list[i]
    thresh = threshold(d3mko)
    binary = d3mko > thresh



    ax[i].imshow(binary, cmap=plt.cm.gray)
    ax[i].set_title(f'{threshold.__name__}')
    ax[i].axis('off')

plt.show()

In [ ]:
small_files

# TFP

In [ ]:
d3tfp = io.imread(small_files[1])


thresh = threshold_otsu(d3tfp)
binary = d3tfp > thresh
fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(d3tfp, cmap=plt.cm.gray)
ax[0].set_title('Day 3 tfp')
ax[0].axis('off')

ax[1].hist(d3mko.ravel(), bins=256)
ax[1].set_title('Hisotgram')
ax[1].axvline(thresh, color='r')

ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Threshold')
ax[2].axis('off')

plt.show()
#io.imsave(os.path.join(os.getcwd(), 'export', 'thresh_tfp.tif'), img_as_uint(binary))

## Watershed segmentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max



In [ ]:
distance = ndi.distance_transform_edt(binary)

In [ ]:
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=binary)

In [ ]:
markers = ndi.label(local_maxi)[0]

In [ ]:
labels = watershed(-distance, markers, mask=binary)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(9,3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(binary, cmap=plt.cm.gray, interpolation='nearest')
ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest')
for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
small_files

In [ ]:
stack = io.imread(small_files[8])
for plane, channel in enumerate(stack):
    thresh = threshold_otsu(channel)
    binary = channel > thresh
    fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(channel, cmap=plt.cm.gray)
    ax[0].set_title('channel')
    ax[0].axis('off')

    ax[1].hist(stack.ravel(), bins=256)
    ax[1].set_title('Hisotgram')
    ax[1].axvline(thresh, color='r')

    ax[2].imshow(binary, cmap=plt.cm.gray)
    ax[2].set_title('Threshold')
    ax[2].axis('off')

    plt.show()
    

In [ ]:
stack.shape

In [ ]:
enumerate(stack)

## Running on slightly bigger images

In [ ]:
images

In [ ]:
from skimage.filters import threshold_li, threshold_triangle, threshold_sauvola, threshold_local

In [ ]:
im_list = [io.imread(i) for i in images]

In [ ]:
im_list[0].shape

In [ ]:
for i in range(len(im_list)):
    image = im_list[i]
    name = images[i]
    thresh = threshold_local(image, 77)
    binary = image > thresh
    io.imsave(os.path.join(os.getcwd(), 'export', str(i) + '_local.tif'), img_as_uint(binary))
    

In [ ]:
bigfiles = os.listdir(os.path.join(os.getcwd(), 'big_tifs'))

In [ ]:
bigfiles

In [ ]:
bigfilepaths = [os.path.join(os.getcwd(), 'big_tifs', i) for i in bigfiles]

In [ ]:
bigfilepaths

In [ ]:
bigmko = io.imread(bigfilepaths[2])

In [ ]:
bigmko.shape

In [ ]:
thresh = threshold_otsu(bigmko)
binary = bigmko > thresh
io.imsave(os.path.join(os.getcwd(), 'bigmko_otsu.tif'), img_as_uint(binary))

In [ ]:
bigfilepaths

In [ ]:
for i in range(len(bigfiles)):
    filepath = bigfilepaths[i]
    image = io.imread(bigfilepaths[i])
    thresh = threshold_otsu(image)
    binary = image > thresh
    io.imsave(os.path.join(os.getcwd(), 'export', bigfiles[i]), img_as_uint(binary))

In [ ]:
small_tfp = io.imread(os.path.join(os.getcwd(), 'small_tifs', 'C1-day3.tif'))

In [ ]:
from scipy.ndimage import gaussian_filter
from skimage import img_as_float
from skimage.morphology import reconstruction

In [ ]:
small_tfp_f = img_as_float(small_tfp)

In [ ]:
big_names = os.listdir(os.path.join(os.getcwd(), 'big_tifs', 'zstack'))
big_filepaths = [os.path.join(os.getcwd(), 'big_tifs', 'zstack', i) for i in big_names]
big_stacks = [img_as_float(io.imread(i)) for i in big_filepaths]

In [ ]:
io.imshow(small_tfp_f)

In [ ]:
small_tfp_g = gaussian_filter(small_tfp_f, 1)

In [ ]:
io.imshow(small_tfp_g)

In [ ]:
seed = np.copy(small_tfp_g)

In [ ]:
seed[1:-1,1:-1] = small_tfp_g.min()

In [ ]:
mask = small_tfp_g

In [ ]:
dilated = reconstruction(seed, mask, method='dilation')

In [ ]:
io.imshow(dilated)

In [ ]:
subtracted_tfp = small_tfp_g - dilated

In [ ]:
io.imshow(subtracted_tfp)

In [ ]:
thresh = threshold_otsu(subtracted_tfp)
binary = subtracted_tfp > thresh
io.imshow(binary)

In [ ]:
io.imsave(os.path.join(os.getcwd(), 'export', 'dilated.tif'), img_as_uint(dilated))

In [ ]:
io.imsave(os.path.join(os.getcwd(), 'export', 'sub_tfp_from_dilation.tif'), img_as_uint(subtracted_tfp))
#io.imsave(os.path.join(os.getcwd(), 'export', 'thresh_from_dilation.tif'), img_as_uint(binary))